In [37]:
import numpy as np
import pandas as pd

In [38]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (1).csv


In [39]:
df= pd.read_csv('dataset.csv')
df['card'] = df['card'].map({'no': 0, 'yes': 1})
df['owner'] = df['owner'].map({'no': 0, 'yes': 1})
df['selfemp'] = df['selfemp'].map({'no': 0, 'yes': 1})
dataset = df.values

### **Decision Tree**

In [40]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [41]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,1:], dataset[:,0]
        num_samples, num_features = np.shape(X)

        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(1,num_features+1):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, 0], dataset_left[:, 0], dataset_right[:, 0]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain

    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        Y = Y.reshape(-1, 1)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

In [42]:
Y = dataset[:, 0]
X = dataset[:, 1:]
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X,Y)
classifier.print_tree()

X_11 <= 0.0 ? 0.033157475461424724
 left:X_10 <= 44.0 ? 0.003298522984734542
  left:X_10 <= 33.0 ? 0.004024021559212665
    left:X_10 <= 32.0 ? 0.0010469977036807743
        left:0.0
        right:0.0
    right:4.0
  right:1.0
 right:X_10 <= 29.0 ? 0.0023648091281056882
  left:X_10 <= 28.0 ? 0.0006428700258530595
    left:X_10 <= 27.0 ? 3.585182671825282e-05
        left:0.0
        right:0.0
    right:1.0
  right:2.0


# **Random Forest**

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
Y = dataset[:, 0]
X = dataset[:, 1:]
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf.fit(X, Y)
Y_pred = rf.predict(X)
print("Training Accuracy:", accuracy_score(Y, Y_pred))

Training Accuracy: 0.9833206974981046
